In [ ]:
import pandas as pd
from primitive_objects import Authors, ArxivPaper, SemSchPaper
import numpy as np
from tqdm import tqdm
import json

In [ ]:
sem_sch_papers_df = pd.read_pickle("../data/sem_sch_papers.pickle")
arxiv_papers_df = pd.read_pickle("../data/arxiv_paper_ids.pickle")
authors_df = pd.read_pickle("../data/sem_sch_authors.pickle")

In [ ]:
author_primary_key = authors_df[["name", "paperCount", "citationCount","hIndex"]].drop_duplicates()
author_primary_key.index = range(author_primary_key.shape[0])
author_primary_key["author_unq_id"] = range(author_primary_key.shape[0])
authors_df_new = pd.merge(authors_df,author_primary_key, on=["name", "paperCount","citationCount","hIndex"], how="left")
author_ids = authors_df["authorId"].unique()
authors_df_np = authors_df.to_numpy()

In [ ]:
authors = []
for author_id in tqdm(author_ids):
    row_nums = np.where(authors_df_np[:, 0] == author_id)[0]
    df_auth =  authors_df_np[row_nums, :]
    author_name = df_auth[:, 1][0]
    author_aliases = np.unique(df_auth[:, 2]).tolist()
    homepage = df_auth[:, 3][0]
    paper_count = df_auth[:, 4][0]
    citations = df_auth[:, 5][0]
    hindex = df_auth[:, 6][0]
    author_name_query = df_auth[:, 7][0]
    author = Authors(
        author_id = author_id, 
        author_name = author_name, 
        author_aliases = author_aliases, 
        homepage = homepage, 
        paper_count = paper_count, 
        citations = citations, 
        hindex = hindex, 
        author_name_query=author_name_query
        )
    authors.append(author)

# Papers DF

### sem sch papers df

In [ ]:
paper_ids = sem_sch_papers_df.paperId.unique()
sem_sch_papers_df_np = sem_sch_papers_df.to_numpy()

In [ ]:
sem_sch_papers_df = sem_sch_papers_df.sort_values(by="paperId")
sem_sch_papers_df.index = range(sem_sch_papers_df.shape[0])

In [ ]:
papers = np.zeros((len(paper_ids))).tolist()
prev_id = sem_sch_papers_df.paperId.values[0]
paper_index = 0
for row in tqdm(sem_sch_papers_df.iterrows()):
    curr_id = row[1]["paperId"]
    if row[0] == 0:
        paper = SemSchPaper(
            curr_id,
            row[1]["title"],
            [row[1]["authorId"]],
            row[1]["abstract"],
            row[1]["year"],
            row[1]["referenceCount"],
            row[1]["citationCount"],
            row[1]["influentialCitationCount"],
            row[1]["isOpenAccess"]
        )
    else:
        if curr_id == prev_id:
            
            paper.authors.append(row[1]["authorId"])
        else:
            papers[paper_index] = paper
            paper_index +=1
            paper = SemSchPaper(
                curr_id,
                row[1]["title"],
                [row[1]["authorId"]],
                row[1]["abstract"],
                row[1]["year"],
                row[1]["referenceCount"],
                row[1]["citationCount"],
                row[1]["influentialCitationCount"],
                row[1]["isOpenAccess"]
            )
    prev_id = curr_id

In [ ]:
paper_dict_keys = ["id", "title", "authors", "abstract", "category", "year",
"reference_count","citation_count", "influential_paper_citations","is_open_access"]
author_dict_keys = ["id", "name", "aliases", "homepage", "paper_count", "citations",
"hindex", "name_query", "papers"]


In [10]:
dd = {"a": 1}

In [11]:
if dd.get("a"):
    print("tes")

tes


In [ ]:
paper_dict = {}

for paper in tqdm(papers):
    if isinstance(paper, SemSchPaper):
        local_dict = {}
        paper_id = paper.id
        for k in paper_dict_keys:
            k_v = getattr(paper, k)
            local_dict[k] = k_v
        paper_dict[paper_id] = local_dict

    

In [ ]:

author_dict = {}
for author in tqdm(authors):
    if isinstance(author, Authors):
        local_dict = {}
        author_id = author.id
        for k in author_dict_keys:
            try:
                k_v = getattr(author, k)
            except:
                k_v = None
            local_dict[k] = k_v
        author_dict[author_id] = local_dict

In [ ]:
with open("../data/papers.json", "w") as f:
    json.dump(paper_dict, f)

In [ ]:
with open("../data/authors.json", "w") as f:
    json.dump(author_dict, f)

In [ ]:
with open("../data/all_authors.txt", "w") as f:
    f.write("\n".join(author_ids.tolist()))

with open("../data/all_papers.txt", "w") as f:
    f.write("\n".join(paper_ids.tolist()))

# Real files

In [1]:
import json
import pandas as pd
from primitive_objects import Authors, SemSchPaper
import numpy as np
from tqdm import tqdm
import json
import re
import langid

In [2]:
with open("../data/all_authors.txt", 'r') as f:
    author_ids = f.readlines()
with open("../data/all_papers.txt", 'r') as f:
    paper_ids = f.readlines()
author_ids = [f.replace("\n","") for f in author_ids]
paper_ids = [f.replace("\n","") for f in paper_ids]

with open("../data/papers.json", 'r') as f:
    cache_papers = json.load(f)

with open("../data/authors.json", 'r') as f:
    cache_authors = json.load(f)
     

In [3]:
paper_keys = list(cache_papers.keys())

In [4]:
paper_dict = cache_papers["00000ea534600af26a435c739e17bbaf460ee485"]

In [5]:
SemSchPaper(**paper_dict)

In [6]:
paper_dict.keys()

dict_keys(['id', 'title', 'authors', 'abstract', 'category', 'year', 'reference_count', 'citation_count', 'influential_paper_citations', 'is_open_access'])

In [ ]:
list(cache_papers.keys())[0]

In [ ]:
authors = []
for author_id in author_ids:
    if cache_authors.get(author_id):
        cache_dict = cache_authors[author_id]
        author = Authors(cache_file=cache_dict)
    authors.append(author)
    

In [ ]:
papers = []
for paper_id in paper_ids:
    if cache_papers.get(paper_id):
        cache_dict = cache_papers[paper_id]
        paper = SemSchPaper(cache_file=cache_dict)
    papers.append(paper)

In [ ]:
from transformers import AutoTokenizer, OPTForCausalLM
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-1.3b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-1.3b", device_map="auto")

In [ ]:
entities_suffix = "\n\nWhat scientific entities are mentioned in the abstract above?\n\n"
summary_suffix = "\n\nTLDR:"
split_str= "What scientific entities are mentioned in the abstract above\?\n\n"

In [ ]:
from tqdm import tqdm

In [ ]:
BATCH_SIZE = 512

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for paper in tqdm(papers):
    abstract = paper.abstract
    
    if (abstract is not None) and (langid.classify(abstract)[0] == 'en'):
        abstract = " ".join(abstract.split(" ")[:300])
        input_text = abstract + entities_suffix
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
        outputs = model.generate(input_ids, max_length=160)
        output_text = tokenizer.decode(outputs[0])
        match = re.split(split_str,output_text)
        categories = [f.replace("</s>","") for f in match[-1].split(", ")]
        paper.category = categories
            

In [ ]:
abstract